In [51]:
import tensorflow as tf 

tf.test.gpu_device_name()

''

In [48]:
#!python
#!/usr/bin/env python
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt

from skimage import io, color, filters
from skimage.transform import resize, rotate
from PIL import Image
from tensorflow import keras
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


In [3]:
# zappostesttrain_dir = '../data/ut-zap50k-data/train-test-splits'
# x = loadmat(zappostesttrain_dir)

In [4]:
# type(x)

In [5]:
# x.keys()

In [6]:
# train_index = x['trainIndexAll']
# test_index = x['testIndexAll']

In [7]:
# train_index[0][0][0].shape

In [8]:
# zapposimagepath_dir = '../data/ut-zap50k-data/image-path.mat'

In [9]:
# imagepathdata = loadmat(zapposimagepath_dir)

In [10]:
# imagepathdata

In [11]:
# imagepathdata.get('imagepath')[1000][0][0]

In [12]:
# image = Image.open('../data/ut-zap50k-data/ut-zap50k-images-square/Boots/Ankle/A. Testoni/7965307.5291.jpg')

In [13]:
# image.show()

In [14]:
# np.asarray(image)

In [15]:
# image.getpixel((0,0))

In [16]:
zapposlabels_dir = '../data/ut-zap50k-data/zappos-labels.mat'
zapposlabels = loadmat(zapposlabels_dir)

In [17]:
# dir(zapposlabels)

In [18]:
zapposlabels.keys()

dict_keys(['__header__', '__version__', '__globals__', 'mturkOrder', 'mturkEqual'])

In [19]:
# #build pipeline as if for one image, then extend it with either function or class
# class imagePipeline:
#     def __init__(self,imagepath):
#         self.imagepath = imagepath
#         self.image = Image.open(imagepath)
#     def as_array(self):
#         return np.asarray(self.image)

In [20]:
# img_arrays = []
# for i in imagepathdata.get('imagepath'):
#     imagepath = '../data/ut-zap50k-data/ut-zap50k-images-square/' + i[0][0]
#     image = imagePipeline(imagepath).as_array()
#     img_arrays.append(image)

In [21]:
import os
import shutil
def sub_subfolder(mainpath,subpath,catlist): #subpath i.e. "boot" ... catlist = [over knee, etc...]
    subs = []
    for i in catlist:
        folder = mainpath + subpath + i
        subfolders = [f.path for f in os.scandir(folder) if f.is_dir()]
        for sub in subfolders:
            for f in os.listdir(sub):
                src = os.path.join(sub, f)
                dst = os.path.join(folder, f)
                shutil.move(src, dst)
            shutil.rmtree(sub)
                


In [22]:
main_path = '../data/ut-zap50k-data/ut-zap50k-images-square/'
bootcat_list = os.listdir('../data/ut-zap50k-data/ut-zap50k-images-square/Boots')
sub_subfolder(main_path,'Boots/',bootcat_list)

In [23]:
sandalcat_list = os.listdir('../data/ut-zap50k-data/ut-zap50k-images-square/Sandals')
sub_subfolder(main_path,'Sandals/',sandalcat_list)

In [24]:
shoecat_list = os.listdir('../data/ut-zap50k-data/ut-zap50k-images-square/Shoes')
sub_subfolder(main_path,'Shoes/',shoecat_list)

In [25]:
slippercat_list = os.listdir('../data/ut-zap50k-data/ut-zap50k-images-square/Slippers')
sub_subfolder(main_path,'Slippers/',slippercat_list)

In [26]:
main_path2 = '../data/ut-zap50k-data/'
imagescat_list = os.listdir('../data/ut-zap50k-data/ut-zap50k-images-square')
sub_subfolder(main_path2,'ut-zap50k-images-square/',imagescat_list)

In [29]:
directory = '../data/ut-zap50k-data/ut-zap50k-images-square/'
img_prepocessing = keras.preprocessing.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
)

Found 50031 files belonging to 4 classes.


In [36]:
img_prepocessing.class_names

['Boots', 'Sandals', 'Shoes', 'Slippers']

In [42]:
root_dir = '../data/ut-zap50k-data/ut-zap50k-images-square'
Cls_boots = '/Boots'
Cls_sandals = '/Sandals'
Cls_shoes = '/Shoes'
Cls_slippers = '/Slippers'
Cls_lst = [Cls_boots,Cls_sandals,Cls_shoes,Cls_slippers]

In [38]:
os.makedirs(root_dir +'/train' + Cls_boots)
os.makedirs(root_dir +'/train' + Cls_sandals)
os.makedirs(root_dir +'/train' + Cls_shoes)
os.makedirs(root_dir +'/train' + Cls_slippers)

In [39]:
os.makedirs(root_dir +'/val' + Cls_boots)
os.makedirs(root_dir +'/val' + Cls_sandals)
os.makedirs(root_dir +'/val' + Cls_shoes)
os.makedirs(root_dir +'/val' + Cls_slippers)

In [40]:
os.makedirs(root_dir +'/test' + Cls_boots)
os.makedirs(root_dir +'/test' + Cls_sandals)
os.makedirs(root_dir +'/test' + Cls_shoes)
os.makedirs(root_dir +'/test' + Cls_slippers)

In [45]:
for i in Cls_lst:
    src = "../data/ut-zap50k-data/ut-zap50k-images-square"+i # Folder to copy images from

    allFileNames = os.listdir(src)
    np.random.shuffle(allFileNames)
    train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                              [int(len(allFileNames)*0.7), int(len(allFileNames)*0.85)])
    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]
    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Validation: ', len(val_FileNames))
    # Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, "../data/ut-zap50k-data/ut-zap50k-images-square/train"+i)
    for name in val_FileNames:
        shutil.copy(name, "../data/ut-zap50k-data/ut-zap50k-images-square/val"+i)
    for name in test_FileNames:
        shutil.copy(name, "../data/ut-zap50k-data/ut-zap50k-images-square/test"+i)

Total images:  12834
Training:  8983
Validation:  1925
Total images:  5741
Training:  4018
Validation:  861
Total images:  30173
Training:  21121
Validation:  4526
Total images:  1283
Training:  898
Validation:  192


In [47]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        '../data/ut-zap50k-data/ut-zap50k-images-square/train',
        target_size=(136, 102),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        '../data/ut-zap50k-data/ut-zap50k-images-square/val',
        target_size=(136, 102),
        batch_size=32,
        class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
        '../data/ut-zap50k-data/ut-zap50k-images-square/test',
        target_size=(136, 102),
        batch_size=32,
        class_mode='categorical')

Found 35020 images belonging to 4 classes.
Found 7504 images belonging to 4 classes.
Found 7507 images belonging to 4 classes.


In [49]:
#what loss functions / filters/ kernel size to use
def define_model(nb_filters, kernel_size, input_shape, pool_size):
    model = Sequential() # model is a linear stack of layers (don't change)
    # note: the convolutional layers and dense layers require an activation function
    # see https://keras.io/activations/
    # and https://en.wikipedia.org/wiki/Activation_function
    # options: 'linear', 'sigmoid', 'tanh', 'relu', 'softplus', 'softsign'
    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]),
                        padding='valid', 
                        input_shape=input_shape)) #first conv. layer  KEEP
    model.add(Activation('relu')) # Activation specification necessary for Conv2D and Dense layers
    model.add(Conv2D(nb_filters, (kernel_size[0], kernel_size[1]), padding='valid')) #2nd conv. layer KEEP
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size)) # decreases size, helps prevent overfitting
    model.add(Dropout(0.5)) # zeros out some fraction of inputs, helps prevent overfitting
    model.add(Flatten()) # necessary to flatten before going into conventional dense layer  KEEP
    print('Model flattened out to ', model.output_shape)
    # now start a typical neural network
    model.add(Dense(32)) # (only) 32 neurons in this layer, really?   KEEP
    model.add(Activation('relu'))
    model.add(Dropout(0.5)) # zeros out some fraction of inputs, helps prevent overfitting
    model.add(Dense(nb_classes)) # 10 final nodes (one for each class)  KEEP
    model.add(Activation('softmax')) # softmax at end to pick between classes 0-9 KEEP
    
    # many optimizers available, see https://keras.io/optimizers/#usage-of-optimizers
    # suggest you KEEP loss at 'categorical_crossentropy' for this multiclass problem,
    # and KEEP metrics at 'accuracy'
    # suggest limiting optimizers to one of these: 'adam', 'adadelta', 'sgd'
    model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
    return model

In [28]:
#going to need speficic steps for epochs, steps_per_epoch, validation_steps
#look up early-stopping callback (to help set really high number for epochs so it stops once it reaches the best)
model.fit(
        train_generator,
        steps_per_epoch=2000,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800)